<a href="https://colab.research.google.com/github/rafiulbiswas/ArAIEval_ArabicNLP24_Task_1/blob/main/HallucinationinGPT_4o_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3f2cb27c2195e1671b5c84cd2abdb777980e6dea9aeda8982937f55eefc3246d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [2]:
import os
import sys
import datetime
import re
import requests
import pandas as pd
import numpy as np
import torch
from typing import List, Dict, Tuple, Optional
import time
from datetime import datetime
import nltk
nltk.download('punkt_tab')
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Import your existing RAG components
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from sentence_transformers import SentenceTransformer, SimilarityFunction
from sklearn.preprocessing import LabelEncoder

# Evaluation metrics
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# ─── FANAR API CONFIGURATION ───────────────────────────────────────────────
API_KEY = "P9ghKGxsxZLDFZs4smB1BpRhXwB4NmSR"
API_URL = "https://api.fanar.qa/v1/chat/completions"

# ─── CONFIG (Your existing paths) ──────────────────────────────────────────
DRIVE = "/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG"
ORIG_CSV = os.path.join(DRIVE, "Datasets/MyDataset.csv")
MARBERT_PATH = os.path.join(DRIVE, "arabic_dialect_model")
LABEL_ENCODER_FN = os.path.join(DRIVE, "label_encoder_classes.npy")

# ─── DEVICE ────────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"


In [12]:
from openai import OpenAI
from typing import List

class OpenRouterAPI:
    """OpenRouter API client for Arabic dialectal text generation using GPT-4o-mini"""

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )

    def test_connection(self) -> bool:
        """Test the OpenRouter connection"""
        print("🧪 Testing OpenRouter connection...")
        try:
            test_completion = self.client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[{"role": "user", "content": "Say 'Hello, OpenRouter!' in Arabic"}],
                max_tokens=50
            )
            print(f"✅ Connection successful: {test_completion.choices[0].message.content}")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False

    def generate_answer(self, question: str, dialect: str, context: str = None,
                       temperature: float = 0.3, max_tokens: int = 300) -> str:
        """Generate answer using OpenRouter GPT-4o-mini with optional RAG context"""

        # Create system prompt based on dialect
        system_prompt = f"""You are an intelligent assistant specialized in answering questions in a specific Arabic dialect. Follow these instructions carefully:

Dialect Information: You must answer in the dialect: {dialect}

Instructions:

Answer using only information from the provided context.

If the context is insufficient, say:
-I don't have enough information in the provided context to fully answer this question.

-Do not use your pre-trained information to fill in the missing context.

-Answer only the question posed, and your answer must be concise and relevant.

-Direct answer first.

=Supporting details with quotes.

-Any limitations or gaps in the available information."""

        # Create user prompt
        if context:
            user_prompt = f"""
أنت مساعد ذكي مخصص لتوليد بيانات نصية باللهجة العربية المطلوبة، باستخدام المعلومات المقدّمة في السياقالسؤال: {question} .

📌 **المهمة**: إنشاء جملة أو حوار أو فقرة باللهجة: **{dialect}**
📚 **السياق المرجعي**: {context}

📝 **تعليمات هامة**:
1. يجب أن تعتمد على المعلومات الموجودة في السياق فقط.
2. لا تضف معلومات خارجية أو منطق غير مدعوم من السياق.
3. استخدم كلمات وتراكيب من اللهجة المحددة فقط، وتجنب الفصحى ما أمكن.
4. اجعل النص طبيعيًا وسهل الفهم كما لو أن متحدثًا أصليًا باللهجة قاله.

🗣️ **أمثلة على اللهجات**:
- **المصرية**: "إنت عامل إيه؟"، "ليه كده؟"
- **الخليجية**: "شلونك؟"، "ليش تأخرت؟"
- **الشامية**: "شو بدك؟"، "يلا نروح"
- **المغاربية**: "كيداير؟"، "بزاف عليك"

✏️ **الناتج المطلوب**:
رجاءً ولّد النص باللهجة المحددة فقط، اعتمادًا على المعلومات الموجودة في السياق أعلاه.

ابدأ التوليد الآن 👇
"""
        else:
            user_prompt = f"""السؤال: {question}

تعليمات:
1. أجب باللهجة المحددة: {dialect}
2. إذا كان السؤال يحتاج معلومات محددة مش متوفرة عندي، قول:
  "هالسؤال محتاج معلومات محددة مش متوفرة في معرفتي الحالية. بنصح تدور على مصادر حديثة أو تقدم سياق إضافي."
3. إذا كنت مش متأكد من دقة المعلومة، اذكر مستوى الثقة
4. استخدم تعبيرات اللهجة المناسبة في إجابتك

مستوى الثقة في الإجابة: [عالي/متوسط/منخفض]
السبب: [اذكر سبب مستوى الثقة]

الإجابة:"""

        try:
            completion = self.client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens,
                frequency_penalty=0,
                presence_penalty=0,
                top_p=1.0
            )

            return completion.choices[0].message.content.strip()

        except Exception as e:
            print(f"Exception in OpenRouter API call: {e}")
            return f"Error: {str(e)}"

    def generate_without_rag(self, question: str, dialect: str) -> str:
        """Generate answer without RAG context"""
        return self.generate_answer(question, dialect, context=None)

    def generate_with_rag(self, question: str, dialect: str, retrieved_context: List[str]) -> str:
        """Generate answer with RAG context"""
        # Combine retrieved sentences into context
        context = "\n".join([f"- {sent}" for sent in retrieved_context])
        return self.generate_answer(question, dialect, context=context)


In [13]:
class RAGPipeline:
    """Your existing RAG pipeline - adapted from your code"""

    def __init__(self):
        # Load dialect classifier
        self.tok_dial = AutoTokenizer.from_pretrained(MARBERT_PATH)
        self.mdl_dial = AutoModelForSequenceClassification.from_pretrained(MARBERT_PATH).to(device)
        self.le = LabelEncoder()
        self.le.classes_ = np.load(LABEL_ENCODER_FN, allow_pickle=True)

        # Replace this line in your RAGPipeline.__init__ method:
        # OLD:
        # self.embed_model = SentenceTransformer(
        #     "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        #     device=device
        # )

        # NEW - Recommended Arabic embedding model:
        print("🔄 Loading Arabic embedding model...")

        # Try Arabic models in order of preference
        arabic_models = [
            #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  # Best for Arabic
           # "intfloat/multilingual-e5-large",  # Latest multilingual model
           # "sentence-transformers/distiluse-base-multilingual-cased",  # Good performance
            "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # Your original fallback
        ]

        self.embed_model = None
        for model_name in arabic_models:
            try:
                print(f"📥 Attempting to load: {model_name}")
                self.embed_model = SentenceTransformer(model_name, device=device)

                # Test with Arabic text
                test_text = "هذا اختبار للنموذج العربي"
                test_embedding = self.embed_model.encode([test_text], convert_to_numpy=True)

                print(f"✅ Successfully loaded: {model_name}")
                print(f"📐 Embedding dimension: {test_embedding.shape[-1]}")
                break

            except Exception as e:
                print(f"❌ Failed to load {model_name}: {e}")
                continue

        if self.embed_model is None:
            raise RuntimeError("❌ Failed to load any Arabic embedding model")

        # Set similarity function (keep your existing setting)
        self.embed_model.similarity_fn_name = SimilarityFunction.DOT_PRODUCT

        # Precomputed embeddings will be loaded later
        self.dialect_embeddings = {}
        self.dialect_sentences = {}

    def clean_text(self, text: str) -> str:
        """Clean text (from your code)"""
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"[^\w\s\u0600-\u06FF]", "", text)
        return text.strip()

    def normalize_text(self, text: str) -> str:
        """Normalize Arabic text (from your code)"""
        text = re.sub(r"[إأآا]", "ا", text)
        text = re.sub(r"[ى]", "ي", text)
        text = re.sub(r"[ؤئ]", "ء", text)
        return text

    def predict_dialect(self, text: str) -> str:
        """Predict dialect using MARBERT"""
        toks = self.tok_dial(
            text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=128
        ).to(device)
        idx = self.mdl_dial(**toks).logits.argmax(-1).item()
        return self.le.inverse_transform([idx])[0]

    def precompute_embeddings(self, df: pd.DataFrame):
        """Precompute embeddings by dialect"""
        self.dialect_embeddings, self.dialect_sentences = {}, {}
        df = df.dropna(subset=["Sentence", "Dialect"]).copy()
        df["Sentence"] = df["Sentence"].astype(str).map(self.clean_text).map(self.normalize_text)

        for d in df["Dialect"].unique():
            sents = df.loc[df["Dialect"] == d, "Sentence"].tolist()
            if not sents:
                continue
            embs = self.embed_model.encode(
                sents,
                convert_to_numpy=True,
                normalize_embeddings=True,
                batch_size=64
            ).astype("float32")
            self.dialect_embeddings[d] = embs
            self.dialect_sentences[d] = sents

    def retrieve_top_k(self, query: str, top_k: int = 3) -> Tuple[str, List[str]]:
        """Retrieve top-k similar sentences"""
        # Predict dialect
        dialect = self.predict_dialect(query)

        # Get embeddings for that dialect
        embs = self.dialect_embeddings.get(dialect)
        sents = self.dialect_sentences.get(dialect, [])

        if embs is None or len(sents) == 0:
            return dialect, []

        # Encode query
        query_cleaned = self.normalize_text(self.clean_text(query))
        q_emb = self.embed_model.encode(
            [query_cleaned],
            convert_to_numpy=True,
            normalize_embeddings=True
        ).astype("float32")

        # Calculate similarities
        sims = self.embed_model.similarity(q_emb, embs)[0]

        # Get top-k
        top_n = min(top_k, sims.shape[0])
        idxs = np.argpartition(-sims, top_n-1)[:top_n]
        idxs = idxs[np.argsort(-sims[idxs])]

        # Return sentences only (not scores for context)
        retrieved_sentences = [sents[i] for i in idxs]
        return dialect, retrieved_sentences

In [14]:
class HallucinationEvaluator:
    """Enhanced evaluator for hallucination by comparing LLM outputs with and without RAG"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
        self.smoothing = SmoothingFunction().method1

    def arabic_tokenize(self, text: str) -> List[str]:
        """Better Arabic tokenization"""
        # Remove diacritics and normalize
        text = re.sub(r'[\u064B-\u0652\u0670\u0640]', '', text)  # Remove diacritics
        text = re.sub(r'[إأآا]', 'ا', text)  # Normalize alef
        text = re.sub(r'[ى]', 'ي', text)    # Normalize yeh
        # Split by whitespace and punctuation
        tokens = re.findall(r'[\w\u0600-\u06FF]+', text.lower())
        return [t for t in tokens if len(t) > 1]  # Filter short tokens

    def calculate_rouge(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate ROUGE scores with Arabic preprocessing"""
        # Preprocess Arabic text
        ref_processed = ' '.join(self.arabic_tokenize(reference))
        cand_processed = ' '.join(self.arabic_tokenize(candidate))

        if not ref_processed.strip() or not cand_processed.strip():
            return {'rouge1_f': 0.0, 'rouge2_f': 0.0, 'rougeL_f': 0.0}

        scores = self.rouge_scorer.score(ref_processed, cand_processed)
        return {
            'rouge1_f': scores['rouge1'].fmeasure,
            'rouge2_f': scores['rouge2'].fmeasure,
            'rougeL_f': scores['rougeL'].fmeasure
        }

    def calculate_bleu_scores(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calculate BLEU-1, BLEU-2, BLEU-3, BLEU-4 scores with Arabic tokenization"""
        try:
            ref_tokens = self.arabic_tokenize(reference)
            cand_tokens = self.arabic_tokenize(candidate)

            if not ref_tokens or not cand_tokens:
                return {
                    'bleu1': 0.0,
                    'bleu2': 0.0,
                    'bleu3': 0.0,
                    'bleu4': 0.0,
                    'bleu_cumulative': 0.0
                }

            # Calculate individual BLEU scores
            bleu1 = sentence_bleu([ref_tokens], cand_tokens, weights=(1.0, 0, 0, 0), smoothing_function=self.smoothing)
            bleu2 = sentence_bleu([ref_tokens], cand_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=self.smoothing)
            bleu3 = sentence_bleu([ref_tokens], cand_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=self.smoothing)
            bleu4 = sentence_bleu([ref_tokens], cand_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=self.smoothing)

            # Standard cumulative BLEU-4
            bleu_cumulative = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=self.smoothing)

            return {
                'bleu1': bleu1,
                'bleu2': bleu2,
                'bleu3': bleu3,
                'bleu4': bleu4,
                'bleu_cumulative': bleu_cumulative
            }
        except Exception as e:
            print(f"BLEU calculation error: {e}")
            return {
                'bleu1': 0.0,
                'bleu2': 0.0,
                'bleu3': 0.0,
                'bleu4': 0.0,
                'bleu_cumulative': 0.0
            }

    def calculate_cosine_similarity(self, text1: str, text2: str, embed_model) -> float:
        """Calculate cosine similarity using embeddings"""
        try:
            emb1 = embed_model.encode([text1], normalize_embeddings=True)
            emb2 = embed_model.encode([text2], normalize_embeddings=True)

            # Calculate cosine similarity
            similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
            return float(similarity)
        except Exception as e:
            print(f"Cosine similarity calculation error: {e}")
            return 0.0

    def calculate_linear_similarity(self, text1: str, text2: str, embed_model) -> float:
        """Calculate linear kernel similarity using embeddings"""
        try:
            emb1 = embed_model.encode([text1], normalize_embeddings=False)  # Don't normalize for linear kernel
            emb2 = embed_model.encode([text2], normalize_embeddings=False)

            # Calculate linear kernel similarity (dot product)
            similarity = linear_kernel(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
            return float(similarity)
        except Exception as e:
            print(f"Linear similarity calculation error: {e}")
            return 0.0

    def calculate_semantic_similarity(self, text1: str, text2: str, embed_model) -> float:
        """Calculate semantic similarity using embeddings (kept for backward compatibility)"""
        return self.calculate_cosine_similarity(text1, text2, embed_model)

    def context_overlap(self, context: List[str], answer: str) -> float:
        """Measure how much the answer uses retrieved context (Arabic-aware)"""
        if not context or not answer:
            return 0.0

        answer_tokens = set(self.arabic_tokenize(answer))
        context_tokens = set()
        for sent in context:
            context_tokens.update(self.arabic_tokenize(sent))

        if not context_tokens or not answer_tokens:
            return 0.0

        overlap = len(answer_tokens.intersection(context_tokens))
        return overlap / len(answer_tokens)

    def calculate_perplexity_difference(self, answer_without_rag: str, answer_with_rag: str) -> float:
        """Calculate a simple proxy for perplexity difference based on text complexity"""
        try:
            # Simple complexity measure: vocabulary diversity
            tokens_without = self.arabic_tokenize(answer_without_rag)
            tokens_with = self.arabic_tokenize(answer_with_rag)

            if not tokens_without or not tokens_with:
                return 0.0

            # Vocabulary diversity ratio
            diversity_without = len(set(tokens_without)) / len(tokens_without) if tokens_without else 0
            diversity_with = len(set(tokens_with)) / len(tokens_with) if tokens_with else 0

            return abs(diversity_with - diversity_without)
        except:
            return 0.0

    def calculate_hallucination_score(self, metrics_without_rag: Dict, metrics_with_rag: Dict,
                                    context_consistency: float) -> float:
        """Calculate composite hallucination score (lower is better)"""
        try:
            # Weight different metrics
            rouge_improvement = metrics_with_rag.get('rouge1_f', 0) - metrics_without_rag.get('rouge1_f', 0)
            bleu_improvement = metrics_with_rag.get('bleu_cumulative', 0) - metrics_without_rag.get('bleu_cumulative', 0)

            # Positive improvements reduce hallucination
            # Low context consistency increases hallucination
            hallucination_score = max(0, 1.0 - (rouge_improvement * 0.4 + bleu_improvement * 0.4 + context_consistency * 0.2))

            return min(1.0, hallucination_score)
        except:
            return 1.0  # Maximum hallucination if calculation fails

    def evaluate_pair(self, question: str, ground_truth: str,
                     answer_without_rag: str, answer_with_rag: str,
                     retrieved_context: List[str], embed_model=None) -> Dict:
        """Evaluate a single question-answer pair with enhanced metrics"""

        # ROUGE scores against ground truth
        metrics_without_rag = self.calculate_rouge(ground_truth, answer_without_rag)
        metrics_with_rag = self.calculate_rouge(ground_truth, answer_with_rag)

        # Enhanced BLEU scores
        bleu_without_rag = self.calculate_bleu_scores(ground_truth, answer_without_rag)
        bleu_with_rag = self.calculate_bleu_scores(ground_truth, answer_with_rag)

        # Add BLEU scores to metrics
        metrics_without_rag.update(bleu_without_rag)
        metrics_with_rag.update(bleu_with_rag)

        # Context consistency (only for RAG)
        context_consistency = self.context_overlap(retrieved_context, answer_with_rag)

        # Similarity metrics (if embedding model available)
        cosine_sim_without_rag = 0.0
        cosine_sim_with_rag = 0.0
        linear_sim_without_rag = 0.0
        linear_sim_with_rag = 0.0

        if embed_model:
            cosine_sim_without_rag = self.calculate_cosine_similarity(ground_truth, answer_without_rag, embed_model)
            cosine_sim_with_rag = self.calculate_cosine_similarity(ground_truth, answer_with_rag, embed_model)
            linear_sim_without_rag = self.calculate_linear_similarity(ground_truth, answer_without_rag, embed_model)
            linear_sim_with_rag = self.calculate_linear_similarity(ground_truth, answer_with_rag, embed_model)

        # Additional metrics
        perplexity_diff = self.calculate_perplexity_difference(answer_without_rag, answer_with_rag)
        hallucination_score = self.calculate_hallucination_score(metrics_without_rag, metrics_with_rag, context_consistency)

        return {
            'question': question,
            'ground_truth': ground_truth,
            'answer_without_rag': answer_without_rag,
            'answer_with_rag': answer_with_rag,
            'retrieved_context': '; '.join(retrieved_context),

            # ROUGE metrics
            'rouge1_without_rag': metrics_without_rag['rouge1_f'],
            'rouge2_without_rag': metrics_without_rag['rouge2_f'],
            'rougeL_without_rag': metrics_without_rag['rougeL_f'],
            'rouge1_with_rag': metrics_with_rag['rouge1_f'],
            'rouge2_with_rag': metrics_with_rag['rouge2_f'],
            'rougeL_with_rag': metrics_with_rag['rougeL_f'],

            # Enhanced BLEU metrics
            'bleu1_without_rag': metrics_without_rag['bleu1'],
            'bleu2_without_rag': metrics_without_rag['bleu2'],
            'bleu3_without_rag': metrics_without_rag['bleu3'],
            'bleu4_without_rag': metrics_without_rag['bleu4'],
            'bleu_cumulative_without_rag': metrics_without_rag['bleu_cumulative'],

            'bleu1_with_rag': metrics_with_rag['bleu1'],
            'bleu2_with_rag': metrics_with_rag['bleu2'],
            'bleu3_with_rag': metrics_with_rag['bleu3'],
            'bleu4_with_rag': metrics_with_rag['bleu4'],
            'bleu_cumulative_with_rag': metrics_with_rag['bleu_cumulative'],

            # Similarity metrics
            'cosine_similarity_without_rag': cosine_sim_without_rag,
            'cosine_similarity_with_rag': cosine_sim_with_rag,
            'linear_similarity_without_rag': linear_sim_without_rag,
            'linear_similarity_with_rag': linear_sim_with_rag,

            # Backward compatibility
            'semantic_sim_without_rag': cosine_sim_without_rag,
            'semantic_sim_with_rag': cosine_sim_with_rag,

            # RAG-specific metrics
            'context_consistency': context_consistency,
            'perplexity_difference': perplexity_diff,
            'hallucination_score': hallucination_score,

            # Improvement scores (positive = RAG better)
            'rouge1_improvement': metrics_with_rag['rouge1_f'] - metrics_without_rag['rouge1_f'],
            'rouge2_improvement': metrics_with_rag['rouge2_f'] - metrics_without_rag['rouge2_f'],
            'rougeL_improvement': metrics_with_rag['rougeL_f'] - metrics_without_rag['rougeL_f'],

            'bleu1_improvement': metrics_with_rag['bleu1'] - metrics_without_rag['bleu1'],
            'bleu2_improvement': metrics_with_rag['bleu2'] - metrics_without_rag['bleu2'],
            'bleu3_improvement': metrics_with_rag['bleu3'] - metrics_without_rag['bleu3'],
            'bleu4_improvement': metrics_with_rag['bleu4'] - metrics_without_rag['bleu4'],
            'bleu_cumulative_improvement': metrics_with_rag['bleu_cumulative'] - metrics_without_rag['bleu_cumulative'],

            'cosine_similarity_improvement': cosine_sim_with_rag - cosine_sim_without_rag,
            'linear_similarity_improvement': linear_sim_with_rag - linear_sim_without_rag,
            'semantic_improvement': cosine_sim_with_rag - cosine_sim_without_rag  # Backward compatibility
        }

    def generate_detailed_report(self, results_df) -> str:
        """Generate a detailed evaluation report"""
        report = []
        report.append("🔍 DETAILED HALLUCINATION EVALUATION REPORT")
        report.append("=" * 60)

        # Basic statistics
        total_samples = len(results_df)
        report.append(f"Total samples evaluated: {total_samples}")

        # ROUGE metrics
        report.append("\n📊 ROUGE METRICS:")
        for metric in ['rouge1', 'rouge2', 'rougeL']:
            without_avg = results_df[f'{metric}_without_rag'].mean()
            with_avg = results_df[f'{metric}_with_rag'].mean()
            improvement = results_df[f'{metric}_improvement'].mean()
            improved_count = (results_df[f'{metric}_improvement'] > 0).sum()

            report.append(f"  {metric.upper()}:")
            report.append(f"    Without RAG: {without_avg:.4f}")
            report.append(f"    With RAG: {with_avg:.4f}")
            report.append(f"    Average improvement: {improvement:.4f}")
            report.append(f"    Samples improved: {improved_count}/{total_samples} ({improved_count/total_samples*100:.1f}%)")

        # BLEU metrics
        report.append("\n📊 BLEU METRICS:")
        for metric in ['bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu_cumulative']:
            without_avg = results_df[f'{metric}_without_rag'].mean()
            with_avg = results_df[f'{metric}_with_rag'].mean()
            improvement = results_df[f'{metric}_improvement'].mean()
            improved_count = (results_df[f'{metric}_improvement'] > 0).sum()

            report.append(f"  {metric.upper()}:")
            report.append(f"    Without RAG: {without_avg:.4f}")
            report.append(f"    With RAG: {with_avg:.4f}")
            report.append(f"    Average improvement: {improvement:.4f}")
            report.append(f"    Samples improved: {improved_count}/{total_samples} ({improved_count/total_samples*100:.1f}%)")

        # Similarity metrics
        report.append("\n📊 SIMILARITY METRICS:")
        for metric in ['cosine_similarity', 'linear_similarity']:
            without_avg = results_df[f'{metric}_without_rag'].mean()
            with_avg = results_df[f'{metric}_with_rag'].mean()
            improvement = results_df[f'{metric}_improvement'].mean()
            improved_count = (results_df[f'{metric}_improvement'] > 0).sum()

            report.append(f"  {metric.upper()}:")
            report.append(f"    Without RAG: {without_avg:.4f}")
            report.append(f"    With RAG: {with_avg:.4f}")
            report.append(f"    Average improvement: {improvement:.4f}")
            report.append(f"    Samples improved: {improved_count}/{total_samples} ({improved_count/total_samples*100:.1f}%)")

        # RAG-specific metrics
        report.append("\n📊 RAG-SPECIFIC METRICS:")
        report.append(f"  Average context consistency: {results_df['context_consistency'].mean():.4f}")
        report.append(f"  Average perplexity difference: {results_df['perplexity_difference'].mean():.4f}")
        report.append(f"  Average hallucination score: {results_df['hallucination_score'].mean():.4f}")

        # Overall assessment
        report.append("\n🎯 OVERALL ASSESSMENT:")
        overall_improved = 0
        for metric in ['rouge1_improvement', 'bleu_cumulative_improvement', 'cosine_similarity_improvement']:
            improved_count = (results_df[metric] > 0).sum()
            overall_improved += improved_count

        avg_overall_improvement = overall_improved / (3 * total_samples)
        report.append(f"  Overall RAG effectiveness: {avg_overall_improvement*100:.1f}%")

        if avg_overall_improvement > 0.6:
            assessment = "🟢 RAG is significantly reducing hallucinations"
        elif avg_overall_improvement > 0.4:
            assessment = "🟡 RAG shows moderate improvement"
        else:
            assessment = "🔴 RAG needs optimization - limited improvement"

        report.append(f"  Assessment: {assessment}")

        return "\n".join(report)

In [ ]:
if __name__ == "__main__":
    # Initialize the API client
    OPENROUTER_API_KEY = "sk-or-v1-your-key-here"  # Replace with your actual key
    client = OpenRouterAPI(OPENROUTER_API_KEY)

    # Test connection
    if not client.test_connection():
        exit(1)

    # Example 1: Generate without RAG
    question = "كيف الطقس اليوم؟"
    dialect = "المصرية"
    response = client.generate_without_rag(question, dialect)
    print(f"Response: {response}")

    # Example 2: Generate with RAG context
    context = ["الطقس اليوم مشمس مع درجة حرارة 25 مئوية", "الرياح خفيفة من الشمال"]
    response_with_rag = client.generate_with_rag(question, dialect, context)
    print(f"Response with RAG: {response_with_rag}")

In [17]:
def main_pipeline_enhanced(dataset_path: str, output_path: str, top_k: int = 3, max_samples: int = None):
    """Enhanced main pipeline for comprehensive hallucination evaluation"""

    print("🚀 Starting Enhanced RAG Hallucination Evaluation Pipeline")
    print(f"⏰ Start time: {datetime.now()}")

    # Load dataset
    print("📊 Loading dataset...")
    df = pd.read_csv(dataset_path)

    # Filter required columns
    required_columns = ['Question', 'Dialect', 'Answer']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Dataset must contain columns: {required_columns}")

    df = df[required_columns].dropna()

    if max_samples:
        df = df.head(max_samples)

    print(f"📈 Loaded {len(df)} samples")

    OPENROUTER_API_KEY = "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"
    #client = OpenRouterAPI(api_key)
    #OPENROUTER_API_KEY = "sk-or-v1-your-key-here"  # Replace with your actual key
    client = OpenRouterAPI(OPENROUTER_API_KEY)
    # Initialize components
    print("🔧 Initializing enhanced components...")
    #fanar = OpenRouterAPI(API_KEY, API_URL)
    rag = RAGPipeline()
    evaluator = HallucinationEvaluator()  # Now uses the enhanced version

    # Load and precompute RAG embeddings
    print("🧮 Precomputing RAG embeddings...")
    rag_df = pd.read_csv(ORIG_CSV)  # Your original dataset for RAG
    rag.precompute_embeddings(rag_df)

    # Process each sample
    print("🔄 Processing samples...")
    results = []

    for idx, row in df.iterrows():
        question = row['Question']
        dialect = row['Dialect']
        ground_truth = row['Answer']

        print(f"Processing {idx+1}/{len(df)}: {question[:50]}...")

        try:
            # 1. Retrieve context using RAG
            detected_dialect, retrieved_context = rag.retrieve_top_k(question, top_k)

            # 2. Generate answer WITHOUT RAG
            answer_without_rag = client.generate_without_rag(question, dialect)
            time.sleep(1)  # Rate limiting

            # 3. Generate answer WITH RAG
            answer_with_rag = client.generate_with_rag(question, dialect, retrieved_context)
            time.sleep(1)  # Rate limiting

            # 4. Enhanced evaluation with all metrics
            result = evaluator.evaluate_pair(
                question, ground_truth, answer_without_rag,
                answer_with_rag, retrieved_context, rag.embed_model
            )
            result['detected_dialect'] = detected_dialect
            result['original_dialect'] = dialect

            results.append(result)

        except Exception as e:
            print(f"❌ Error processing sample {idx}: {e}")
            continue

    # Save results
    print("💾 Saving results...")
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False, encoding='utf-8')

    # Generate and print enhanced summary
    print("\n" + "="*70)
    print("📋 ENHANCED EVALUATION SUMMARY")
    print("="*70)

    # Basic metrics
    print(f"Total samples processed: {len(results_df)}")

    # ROUGE Summary
    print(f"\n🔸 ROUGE METRICS:")
    for metric in ['rouge1', 'rouge2', 'rougeL']:
        without_avg = results_df[f'{metric}_without_rag'].mean()
        with_avg = results_df[f'{metric}_with_rag'].mean()
        improvement = results_df[f'{metric}_improvement'].mean()
        improved_samples = (results_df[f'{metric}_improvement'] > 0).sum()

        print(f"  {metric.upper()}: {without_avg:.3f} → {with_avg:.3f} "
              f"(Δ: {improvement:+.3f}, {improved_samples}/{len(results_df)} improved)")

    # BLEU Summary
    print(f"\n🔸 BLEU METRICS:")
    for metric in ['bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu_cumulative']:
        without_avg = results_df[f'{metric}_without_rag'].mean()
        with_avg = results_df[f'{metric}_with_rag'].mean()
        improvement = results_df[f'{metric}_improvement'].mean()
        improved_samples = (results_df[f'{metric}_improvement'] > 0).sum()

        print(f"  {metric.upper()}: {without_avg:.3f} → {with_avg:.3f} "
              f"(Δ: {improvement:+.3f}, {improved_samples}/{len(results_df)} improved)")

    # Similarity Summary
    print(f"\n🔸 SIMILARITY METRICS:")
    for metric in ['cosine_similarity', 'linear_similarity']:
        without_avg = results_df[f'{metric}_without_rag'].mean()
        with_avg = results_df[f'{metric}_with_rag'].mean()
        improvement = results_df[f'{metric}_improvement'].mean()
        improved_samples = (results_df[f'{metric}_improvement'] > 0).sum()

        print(f"  {metric.upper()}: {without_avg:.3f} → {with_avg:.3f} "
              f"(Δ: {improvement:+.3f}, {improved_samples}/{len(results_df)} improved)")

    # RAG-specific metrics
    print(f"\n🔸 RAG-SPECIFIC METRICS:")
    print(f"  Context consistency: {results_df['context_consistency'].mean():.3f}")
    print(f"  Perplexity difference: {results_df['perplexity_difference'].mean():.3f}")
    print(f"  Hallucination score: {results_df['hallucination_score'].mean():.3f} (lower is better)")

    # Overall assessment
    print(f"\n🎯 HALLUCINATION REDUCTION ANALYSIS")
    print("="*50)

    # Calculate overall effectiveness
    key_metrics = ['rouge1_improvement', 'bleu_cumulative_improvement', 'cosine_similarity_improvement']
    overall_effectiveness = 0

    for metric in key_metrics:
        improved_count = (results_df[metric] > 0).sum()
        effectiveness = improved_count / len(results_df)
        overall_effectiveness += effectiveness
        metric_name = metric.replace('_improvement', '').replace('_', ' ').upper()
        print(f"{metric_name}: {improved_count}/{len(results_df)} samples improved ({effectiveness*100:.1f}%)")

    overall_effectiveness /= len(key_metrics)
    print(f"\nOVERALL RAG EFFECTIVENESS: {overall_effectiveness*100:.1f}%")

    if overall_effectiveness > 0.6:
        assessment = "🟢 RAG is significantly reducing hallucinations"
    elif overall_effectiveness > 0.4:
        assessment = "🟡 RAG shows moderate improvement"
    else:
        assessment = "🔴 RAG needs optimization - limited improvement detected"

    print(f"ASSESSMENT: {assessment}")

    # Top performing and problematic samples
    print(f"\n🔍 SAMPLE ANALYSIS:")

    # Best improvement sample
    best_idx = results_df['rouge1_improvement'].idxmax()
    best_sample = results_df.iloc[best_idx]
    print(f"\n📈 BEST IMPROVEMENT (ROUGE-1: +{best_sample['rouge1_improvement']:.3f}):")
    print(f"Question: {best_sample['question'][:80]}...")
    print(f"ROUGE-1: {best_sample['rouge1_without_rag']:.3f} → {best_sample['rouge1_with_rag']:.3f}")
    print(f"BLEU-4: {best_sample['bleu4_without_rag']:.3f} → {best_sample['bleu4_with_rag']:.3f}")
    print(f"Cosine Sim: {best_sample['cosine_similarity_without_rag']:.3f} → {best_sample['cosine_similarity_with_rag']:.3f}")

    # Worst performance sample
    worst_idx = results_df['rouge1_improvement'].idxmin()
    worst_sample = results_df.iloc[worst_idx]
    print(f"\n📉 NEEDS ATTENTION (ROUGE-1: {worst_sample['rouge1_improvement']:+.3f}):")
    print(f"Question: {worst_sample['question'][:80]}...")
    print(f"ROUGE-1: {worst_sample['rouge1_without_rag']:.3f} → {worst_sample['rouge1_with_rag']:.3f}")
    print(f"BLEU-4: {worst_sample['bleu4_without_rag']:.3f} → {worst_sample['bleu4_with_rag']:.3f}")
    print(f"Cosine Sim: {worst_sample['cosine_similarity_without_rag']:.3f} → {worst_sample['cosine_similarity_with_rag']:.3f}")

    # Generate detailed report
    detailed_report = evaluator.generate_detailed_report(results_df)

    # Save detailed report
    report_path = output_path.replace('.csv', '_detailed_report.txt')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(detailed_report)

    print(f"\n✅ Results saved to: {output_path}")
    print(f"📄 Detailed report saved to: {report_path}")
    print(f"⏰ End time: {datetime.now()}")

# Example usage with enhanced metrics
if __name__ == "__main__":
    dataset_path = "/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/evaluation_dataset.csv"
    output_path = "/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/enhanced_hallucination_evaluation_results.csv"

    main_pipeline_enhanced(
        dataset_path=dataset_path,
        output_path=output_path,
        top_k=5,
        max_samples=100  # Remove this for full dataset
    )


🚀 Starting Enhanced RAG Hallucination Evaluation Pipeline
⏰ Start time: 2025-06-28 13:10:55.640334
📊 Loading dataset...
📈 Loaded 100 samples
🔧 Initializing enhanced components...
🔄 Loading Arabic embedding model...
📥 Attempting to load: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Successfully loaded: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
📐 Embedding dimension: 384
🧮 Precomputing RAG embeddings...
🔄 Processing samples...
Processing 1/100: ممكن تشرحلي ازاي أبدأ بيزنس أونلاين من الصفر؟...
Processing 2/100: قولي يا شاطر، إزاي أعمل شاي بالنعناع مظبوط؟...
Processing 3/100: عايز أعرف أماكن حلوة للخروج في القاهرة، ممكن؟...
Processing 4/100:  لو سمحت، في فرق بين التاكسي والميكروباص في مصر؟...
Processing 5/100:  ينفع تكتبلي مقال قصير عن أهمية التعليم؟...
Processing 6/100:  إزاي أتعلم برمجة وأنا مبتدئ؟...
Processing 7/100: "خدلك حاجة ساقعة"، ده أسلوب رسمي ولا عادي؟...
Processing 8/100: إيه أكتر طريقة مؤدبة أطلب بيها حاجة من واحد معرفوش...
Processing 9/100: "معلش" بتنفع في كل المواقف؟ ولا فيها استسهال؟...
Processing 10/100:  ممكن تخبرني عن أفضل مجمعات تجارية في دبي؟...
Processing 11/100:  كيف أقدر أبدأ مشروعي الخاص بدون راس مال كبير؟...
Processing 12/100: شرايك تكتبلي رسالة شكر رسمية بلهجة خليجية؟...
Processing 13/100:  وين أحسن أماكن ترفيهية للعوائ

## LLM as Judge

In [19]:
from openai import OpenAI
import pandas as pd
import time
import re
from typing import Dict, Optional
from datetime import datetime

class LLMJudgeEvaluator:
    """LLM-as-Judge evaluator using OpenAI client with OpenRouter"""

    def __init__(self, api_key: str, model: str = "google/gemini-2.0-flash-001"):
        self.api_key = api_key
        self.model = model

        # Initialize OpenAI client with OpenRouter
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )

        # Evaluation prompt template
        self.EVALUATION_PROMPT = """### Task Description:
You are provided with:
- An instruction to generate Arabic dialectal text
- A human-written ground truth (reference)
- Two model outputs:
    1. Output without using RAG (baseline)
    2. Output using RAG (retrieval-augmented)

Your tasks:
1. Evaluate how well each model output matches the reference based on the rubric.
2. Give **specific feedback** for each output using the rubric criteria.
3. Assign a score between 1 and 5 to each output.
4. Finally, say which output is better, or if it's a tie.

---

### Question / Instruction:
{question}

### Ground Truth (Reference):
{ground_truth}

### Baseline Output (Without RAG):
{answer_without_rag}

### RAG Output (With RAG):
{answer_with_rag}

---

### Evaluation Rubric:
Evaluate each output according to:
1. **Dialectal Accuracy**: Does it match the dialectal form expected (e.g., Egyptian, Gulf, etc.)?
2. **Content Relevance**: Does it reflect the core idea in the question and align with the reference?
3. **Fluency & Naturalness**: Does the text read like something a native speaker would say?
4. **Completeness**: Is the output sufficient and covers the expected meaning or intent?

**Scoring scale (for each output)**:
- **1**: Poor — fails most criteria
- **2**: Weak — major issues present
- **3**: Fair — partially aligned, but flawed
- **4**: Good — mostly aligns with the reference
- **5**: Excellent — highly similar, natural, and accurate

---

### Output Format:
Please provide your evaluation in the following structured format:

**BASELINE EVALUATION (Without RAG):**
- Dialectal Accuracy: [Brief assessment]
- Content Relevance: [Brief assessment]
- Fluency & Naturalness: [Brief assessment]
- Completeness: [Brief assessment]
- Overall Feedback: [Detailed feedback]
- Score: [1-5]

**RAG EVALUATION (With RAG):**
- Dialectal Accuracy: [Brief assessment]
- Content Relevance: [Brief assessment]
- Fluency & Naturalness: [Brief assessment]
- Completeness: [Brief assessment]
- Overall Feedback: [Detailed feedback]
- Score: [1-5]

**FINAL COMPARISON:**
- Better Output: [Baseline/RAG/Tie]
- Reasoning: [Explanation for the choice]
- Score Difference: [RAG_Score - Baseline_Score]

Begin your evaluation:"""

    def evaluate_single_pair(self, question: str, ground_truth: str,
                           answer_without_rag: str, answer_with_rag: str,
                           max_retries: int = 3) -> Optional[str]:
        """Evaluate a single question-answer pair using LLM-as-judge"""

        # Format the evaluation prompt
        prompt = self.EVALUATION_PROMPT.format(
            question=question,
            ground_truth=ground_truth,
            answer_without_rag=answer_without_rag,
            answer_with_rag=answer_with_rag
        )

        for attempt in range(max_retries):
            try:
                # Use OpenAI client with OpenRouter
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    temperature=0.1,  # Low temperature for consistent evaluation
                    max_tokens=2000
                )

                return completion.choices[0].message.content.strip()

            except Exception as e:
                print(f"❌ Attempt {attempt + 1} failed: {e}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                    continue
                else:
                    print(f"❌ All {max_retries} attempts failed for this sample")
                    return None

        return None

    def parse_evaluation_response(self, response: str) -> Dict:
        """Parse the LLM judge response into structured data"""

        result = {
            'llm_judge_baseline_score': 0,
            'llm_judge_rag_score': 0,
            'llm_judge_better_output': 'Unknown',
            'llm_judge_score_difference': 0,
            'llm_judge_baseline_feedback': '',
            'llm_judge_rag_feedback': '',
            'llm_judge_comparison_reasoning': '',
            'llm_judge_raw_evaluation': response,
            'llm_judge_status': 'success'
        }

        try:
            # Extract baseline score
            baseline_match = re.search(r'BASELINE.*?Score:\s*\[?(\d+)', response, re.DOTALL | re.IGNORECASE)
            if baseline_match:
                result['llm_judge_baseline_score'] = int(baseline_match.group(1))

            # Extract RAG score
            rag_match = re.search(r'RAG.*?Score:\s*\[?(\d+)', response, re.DOTALL | re.IGNORECASE)
            if rag_match:
                result['llm_judge_rag_score'] = int(rag_match.group(1))

            # Extract better output
            better_match = re.search(r'Better Output:\s*\[(.*?)\]', response, re.IGNORECASE)
            if better_match:
                result['llm_judge_better_output'] = better_match.group(1).strip()

            # Extract feedback sections
            baseline_feedback_match = re.search(r'BASELINE.*?Overall Feedback:\s*\[(.*?)\]', response, re.DOTALL | re.IGNORECASE)
            if baseline_feedback_match:
                result['llm_judge_baseline_feedback'] = baseline_feedback_match.group(1).strip()

            rag_feedback_match = re.search(r'RAG.*?Overall Feedback:\s*\[(.*?)\]', response, re.DOTALL | re.IGNORECASE)
            if rag_feedback_match:
                result['llm_judge_rag_feedback'] = rag_feedback_match.group(1).strip()

            # Extract reasoning
            reasoning_match = re.search(r'Reasoning:\s*\[(.*?)\]', response, re.IGNORECASE)
            if reasoning_match:
                result['llm_judge_comparison_reasoning'] = reasoning_match.group(1).strip()

            # Calculate score difference
            result['llm_judge_score_difference'] = result['llm_judge_rag_score'] - result['llm_judge_baseline_score']

        except Exception as e:
            print(f"⚠️ Error parsing evaluation response: {e}")
            result['llm_judge_status'] = 'parsing_error'

        return result

def evaluate_with_llm_judge_openai(question: str, ground_truth: str,
                                 answer_without_rag: str, answer_with_rag: str,
                                 openai_client: OpenAI, model: str = "openai/o1-mini") -> Dict:
    """
    Single function to evaluate using OpenAI client format
    """

    evaluation_prompt = f"""### Task Description:
You are provided with:
- An instruction to generate Arabic dialectal text
- A human-written ground truth (reference)
- Two model outputs:
    1. Output without using RAG (baseline)
    2. Output using RAG (retrieval-augmented)

Your tasks:
1. Evaluate how well each model output matches the reference based on the rubric.
2. Give **specific feedback** for each output using the rubric criteria.
3. Assign a score between 1 and 5 to each output.
4. Finally, say which output is better, or if it's a tie.

---

### Question / Instruction:
{question}

### Ground Truth (Reference):
{ground_truth}

### Baseline Output (Without RAG):
{answer_without_rag}

### RAG Output (With RAG):
{answer_with_rag}

---

### Evaluation Rubric:
Evaluate each output according to:
1. **Dialectal Accuracy**: Does it match the dialectal form expected (e.g., Egyptian, Gulf, etc.)?
2. **Content Relevance**: Does it reflect the core idea in the question and align with the reference?
3. **Fluency & Naturalness**: Does the text read like something a native speaker would say?
4. **Completeness**: Is the output sufficient and covers the expected meaning or intent?

**Scoring scale (for each output)**:
- **1**: Poor — fails most criteria
- **2**: Weak — major issues present
- **3**: Fair — partially aligned, but flawed
- **4**: Good — mostly aligns with the reference
- **5**: Excellent — highly similar, natural, and accurate

---

### Output Format:
Please provide your evaluation in the following structured format:

**BASELINE EVALUATION (Without RAG):**
- Dialectal Accuracy: [Brief assessment]
- Content Relevance: [Brief assessment]
- Fluency & Naturalness: [Brief assessment]
- Completeness: [Brief assessment]
- Overall Feedback: [Detailed feedback]
- Score: [1-5]

**RAG EVALUATION (With RAG):**
- Dialectal Accuracy: [Brief assessment]
- Content Relevance: [Brief assessment]
- Fluency & Naturalness: [Brief assessment]
- Completeness: [Brief assessment]
- Overall Feedback: [Detailed feedback]
- Score: [1-5]

**FINAL COMPARISON:**
- Better Output: [Baseline/RAG/Tie]
- Reasoning: [Explanation for the choice]
- Score Difference: [RAG_Score - Baseline_Score]

Begin your evaluation:"""

    try:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": evaluation_prompt}],
            temperature=0.1,
            max_tokens=2000
        )

        result = completion.choices[0].message.content.strip()

        # Parse scores
        baseline_score = 0
        rag_score = 0
        better_output = "Unknown"

        # Extract baseline score
        baseline_match = re.search(r'BASELINE.*?Score:\s*\[?(\d+)', result, re.DOTALL | re.IGNORECASE)
        if baseline_match:
            baseline_score = int(baseline_match.group(1))

        # Extract RAG score
        rag_match = re.search(r'RAG.*?Score:\s*\[?(\d+)', result, re.DOTALL | re.IGNORECASE)
        if rag_match:
            rag_score = int(rag_match.group(1))

        # Extract better output
        better_match = re.search(r'Better Output:\s*\[(.*?)\]', result, re.IGNORECASE)
        if better_match:
            better_output = better_match.group(1).strip()

        return {
            'llm_judge_baseline_score': baseline_score,
            'llm_judge_rag_score': rag_score,
            'llm_judge_better_output': better_output,
            'llm_judge_score_difference': rag_score - baseline_score,
            'llm_judge_raw_evaluation': result,
            'llm_judge_status': 'success'
        }

    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        return {
            'llm_judge_baseline_score': 0,
            'llm_judge_rag_score': 0,
            'llm_judge_better_output': 'Error',
            'llm_judge_score_difference': 0,
            'llm_judge_raw_evaluation': f"Error: {str(e)}",
            'llm_judge_status': 'failed'
        }

def add_llm_judge_evaluation_openai(csv_path: str, output_path: str,
                                  openai_client: OpenAI, model: str = "openai/o1-mini",
                                  sample_size: int = None):
    """
    Add LLM-as-judge evaluation using OpenAI client format
    """
    print("🚀 Starting LLM-as-Judge Evaluation with OpenAI Client")

    # Load your existing results
    df = pd.read_csv(csv_path)

    # Check required columns
    required_cols = ['question', 'ground_truth', 'answer_without_rag', 'answer_with_rag']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"❌ Missing columns: {missing_cols}")
        return

    # Sample if requested
    if sample_size:
        df = df.head(sample_size)
        print(f"📊 Evaluating sample of {len(df)} rows")
    else:
        print(f"📊 Evaluating full dataset: {len(df)} rows")

    # Add LLM judge columns
    llm_judge_results = []

    for idx, row in df.iterrows():
        print(f"🔄 Evaluating {idx + 1}/{len(df)}: {row['question'][:50]}...")

        # Call LLM judge
        judge_result = evaluate_with_llm_judge_openai(
            question=row['question'],
            ground_truth=row['ground_truth'],
            answer_without_rag=row['answer_without_rag'],
            answer_with_rag=row['answer_with_rag'],
            openai_client=openai_client,
            model=model
        )

        llm_judge_results.append(judge_result)

        print(f"✅ Baseline: {judge_result['llm_judge_baseline_score']}, "
              f"RAG: {judge_result['llm_judge_rag_score']}, "
              f"Better: {judge_result['llm_judge_better_output']}")

        # Rate limiting - adjust based on your API limits
        time.sleep(1.0)

    # Add results to dataframe
    judge_df = pd.DataFrame(llm_judge_results)
    result_df = pd.concat([df, judge_df], axis=1)

    # Save enhanced results
    result_df.to_csv(output_path, index=False, encoding='utf-8')

    # Generate summary
    successful_evals = judge_df[judge_df['llm_judge_status'] == 'success']
    if len(successful_evals) > 0:
        print(f"\n📋 LLM JUDGE SUMMARY:")
        print(f"Successful evaluations: {len(successful_evals)}/{len(df)}")
        print(f"Average Baseline Score: {successful_evals['llm_judge_baseline_score'].mean():.2f}")
        print(f"Average RAG Score: {successful_evals['llm_judge_rag_score'].mean():.2f}")
        print(f"Average Improvement: {successful_evals['llm_judge_score_difference'].mean():.2f}")

        # Win/Loss/Tie counts
        outcome_counts = successful_evals['llm_judge_better_output'].value_counts()
        for outcome, count in outcome_counts.items():
            percentage = (count / len(successful_evals)) * 100
            print(f"{outcome}: {count} ({percentage:.1f}%)")

    print(f"✅ Enhanced results saved to: {output_path}")
    return result_df

# USAGE EXAMPLE FOR YOUR NOTEBOOK:
if __name__ == "__main__":
    # Step 1: Initialize OpenAI client with OpenRouter
    OPENROUTER_API_KEY = "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"  # Replace with your actual key

    openai_client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )

    # Step 2: Test the client (optional)
    print("🧪 Testing OpenRouter connection...")
    try:
        test_completion = openai_client.chat.completions.create(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": "Say 'Hello, OpenRouter!' in Arabic"}],
            max_tokens=50
        )
        print(f"✅ Connection successful: {test_completion.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        exit(1)

    # Step 3: Run LLM judge evaluation
    enhanced_df = add_llm_judge_evaluation_openai(
        csv_path="/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/enhanced_hallucination_evaluation_results.csv",
        output_path="/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/llm_judge_enhanced_results.csv",
        openai_client=openai_client,
        model="openai/o1-mini",  # or "openai/gpt-4o", "anthropic/claude-3.5-sonnet"
        #sample_size=10  # Remove this to evaluate full dataset
    )

    print("🎉 LLM-as-Judge evaluation completed!")

🧪 Testing OpenRouter connection...
✅ Connection successful: Hello, OpenRouter! in Arabic is: مرحبًا، أوبن روتير!
🚀 Starting LLM-as-Judge Evaluation with OpenAI Client
📊 Evaluating full dataset: 100 rows
🔄 Evaluating 1/100: ممكن تشرحلي ازاي أبدأ بيزنس أونلاين من الصفر؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 2/100: قولي يا شاطر، إزاي أعمل شاي بالنعناع مظبوط؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 3/100: عايز أعرف أماكن حلوة للخروج في القاهرة، ممكن؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 4/100:  لو سمحت، في فرق بين التاكسي والميكروباص في مصر؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 5/100:  ينفع تكتبلي مقال قصير عن أهمية التعليم؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 6/100:  إزاي أتعلم برمجة وأنا مبتدئ؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 7/100: "خدلك حاجة ساقعة"، ده أسلوب رسمي ولا عادي؟...
✅ Baseline: 0, RAG: 0, Better: Unknown
🔄 Evaluating 8/100: إيه أكتر طريقة مؤدبة أطلب بيها حاجة من واحد 

In [20]:
import re
from typing import Dict

def parse_evaluation_response_fixed(response: str) -> Dict:
    """Fixed parsing function that handles Arabic text and markdown formatting"""

    result = {
        'llm_judge_baseline_score': 0,
        'llm_judge_rag_score': 0,
        'llm_judge_better_output': 'Unknown',
        'llm_judge_score_difference': 0,
        'llm_judge_baseline_feedback': '',
        'llm_judge_rag_feedback': '',
        'llm_judge_comparison_reasoning': '',
        'llm_judge_raw_evaluation': response,
        'llm_judge_status': 'success'
    }

    try:
        # Extract baseline score - handle both [Score: 5] and Score: 5 formats
        baseline_score_patterns = [
            r'BASELINE.*?Score:\s*\*\*\s*(\d+)\s*\*\*',  # **Score: 5**
            r'BASELINE.*?Score:\s*\[(\d+)\]',              # [Score: 5]
            r'BASELINE.*?Score:\s*(\d+)',                  # Score: 5
            r'BASELINE.*?\*\*Score:\*\*\s*(\d+)',         # **Score:** 5
        ]

        for pattern in baseline_score_patterns:
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                result['llm_judge_baseline_score'] = int(match.group(1))
                break

        # Extract RAG score - handle multiple formats
        rag_score_patterns = [
            r'RAG.*?Score:\s*\*\*\s*(\d+)\s*\*\*',        # **Score: 4**
            r'RAG.*?Score:\s*\[(\d+)\]',                   # [Score: 4]
            r'RAG.*?Score:\s*(\d+)',                       # Score: 4
            r'RAG.*?\*\*Score:\*\*\s*(\d+)',              # **Score:** 4
        ]

        for pattern in rag_score_patterns:
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                result['llm_judge_rag_score'] = int(match.group(1))
                break

        # Extract better output - handle multiple formats
        better_output_patterns = [
            r'Better Output:\s*\*\*\s*(.*?)\s*\*\*',      # **Better Output:** **Baseline**
            r'Better Output:\s*\[(.*?)\]',                 # Better Output: [Baseline]
            r'Better Output:\s*(Baseline|RAG|Tie)',       # Better Output: Baseline
            r'\*\*Better Output:\*\*\s*(.*?)(?:\n|$)',    # **Better Output:** Baseline
        ]

        for pattern in better_output_patterns:
            match = re.search(pattern, response, re.IGNORECASE)
            if match:
                result['llm_judge_better_output'] = match.group(1).strip()
                break

        # Extract baseline feedback - look for Overall Feedback section
        baseline_feedback_patterns = [
            r'BASELINE.*?Overall Feedback:\s*\*\*\s*(.*?)\s*\*\*.*?(?=\*\*RAG|$)',  # **Overall Feedback:** **text**
            r'BASELINE.*?Overall Feedback:\s*(.*?)(?=\n.*?Score:|$)',               # Overall Feedback: text
            r'BASELINE.*?\*\*Overall Feedback:\*\*\s*(.*?)(?=\n.*?\*\*Score|$)',   # **Overall Feedback:** text
        ]

        for pattern in baseline_feedback_patterns:
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                result['llm_judge_baseline_feedback'] = match.group(1).strip()
                break

        # Extract RAG feedback
        rag_feedback_patterns = [
            r'RAG.*?Overall Feedback:\s*\*\*\s*(.*?)\s*\*\*.*?(?=\*\*FINAL|$)',    # **Overall Feedback:** **text**
            r'RAG.*?Overall Feedback:\s*(.*?)(?=\n.*?Score:|$)',                   # Overall Feedback: text
            r'RAG.*?\*\*Overall Feedback:\*\*\s*(.*?)(?=\n.*?\*\*Score|$)',       # **Overall Feedback:** text
        ]

        for pattern in rag_feedback_patterns:
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                result['llm_judge_rag_feedback'] = match.group(1).strip()
                break

        # Extract reasoning
        reasoning_patterns = [
            r'Reasoning:\s*\*\*\s*(.*?)\s*\*\*',          # **Reasoning:** **text**
            r'Reasoning:\s*\[(.*?)\]',                     # Reasoning: [text]
            r'Reasoning:\s*(.*?)(?=\n.*?Score Difference|$)',  # Reasoning: text
            r'\*\*Reasoning:\*\*\s*(.*?)(?=\n|$)',        # **Reasoning:** text
        ]

        for pattern in reasoning_patterns:
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                result['llm_judge_comparison_reasoning'] = match.group(1).strip()
                break

        # Calculate score difference
        if result['llm_judge_rag_score'] and result['llm_judge_baseline_score']:
            result['llm_judge_score_difference'] = result['llm_judge_rag_score'] - result['llm_judge_baseline_score']

        # Validate that we got the essential data
        if result['llm_judge_baseline_score'] == 0 and result['llm_judge_rag_score'] == 0:
            result['llm_judge_status'] = 'parsing_failed'
            print("⚠️ Failed to extract scores from response")

    except Exception as e:
        print(f"⚠️ Error parsing evaluation response: {e}")
        result['llm_judge_status'] = 'parsing_error'

    return result

def test_parsing_with_your_example():
    """Test the parsing function with your actual example"""

    your_example = """**BASELINE EVALUATION (Without RAG):**
- **Dialectal Accuracy:** النص مكتوب باللهجة المصرية العامية بشكل دقيق ومتناسق.
- **Content Relevance:** يتناول خطوات بدء البيزنس الأونلاين بشكل شامل ويتوافق مع الفكرة الأساسية المرجعية.
- **Fluency & Naturalness:** النص سلس وطبيعي، كأنه مُوجه من متحدث أصلي باللهجة المصرية.
- **Completeness:** يقدم معلومات مفصلة وشاملة حول كيفية بدء البيزنس الأونلاين، بما في ذلك تحديد الفكرة، بناء الموقع، والتسويق.
- **Overall Feedback:** إجابة ممتازة ومفصلة تتجاوز المرجع بشكل إيجابي، مما يوفر للقارئ خطوات واضحة وعملية لبدء البيزنس الأونلاين.
- **Score:** 5

**RAG EVALUATION (With RAG):**
- **Dialectal Accuracy:** النص مكتوب باللهجة المصرية لكن استخدام بعض التعابير قد يكون أقل شيوعًا أو طبيعيًا قليلاً.
- **Content Relevance:** يتناول نفس الموضوع ويركز على تحديد الفكرة وفتح وسائل التواصل الاجتماعي، لكنه أقل تفصيلاً من النسخة الأساسية.
- **Fluency & Naturalness:** النص مفهوم وطبيعي إلى حد كبير، لكن بعض الجمل قد تبدو مبسطة أكثر مما هو مطلوب.
- **Completeness:** يغطي الأساسيات ولكنه لا يقدم تفاصيل إضافية أو خطوات عملية كما في النسخة الأساسية.
- **Overall Feedback:** إجابة جيدة توفر معلومات أساسية حول بدء البيزنس الأونلاين، لكنها تفتقر إلى التفصيل والعمق الموجودين في النسخة الأساسية.
- **Score:** 4

**FINAL COMPARISON:**
- **Better Output:** Baseline
- **Reasoning:** النسخة الأساسية تقدم معلومات أكثر تفصيلاً وشمولاً وتكتب باللهجة المصرية بشكل أكثر طلاقة وطبيعية، مما يجعلها أكثر فائدة وملاءمة للسؤال المطروح.
- **Score Difference:** 1"""

    result = parse_evaluation_response_fixed(your_example)

    print("🧪 TESTING PARSING WITH YOUR EXAMPLE:")
    print(f"Baseline Score: {result['llm_judge_baseline_score']}")
    print(f"RAG Score: {result['llm_judge_rag_score']}")
    print(f"Better Output: {result['llm_judge_better_output']}")
    print(f"Score Difference: {result['llm_judge_score_difference']}")
    print(f"Status: {result['llm_judge_status']}")
    print(f"Baseline Feedback: {result['llm_judge_baseline_feedback'][:100]}...")
    print(f"RAG Feedback: {result['llm_judge_rag_feedback'][:100]}...")
    print(f"Reasoning: {result['llm_judge_comparison_reasoning'][:100]}...")

    return result

# Test with your example
if __name__ == "__main__":
    test_result = test_parsing_with_your_example()

# Updated evaluation function with fixed parsing
def evaluate_with_llm_judge_openai_fixed(question: str, ground_truth: str,
                                        answer_without_rag: str, answer_with_rag: str,
                                        openai_client, model: str = "openai/o1-mini") -> Dict:
    """
    Updated evaluation function with fixed parsing
    """

    evaluation_prompt = f"""### Task Description:
You are provided with:
- An instruction to generate Arabic dialectal text
- A human-written ground truth (reference)
- Two model outputs:
    1. Output without using RAG (baseline)
    2. Output using RAG (retrieval-augmented)

Your tasks:
1. Evaluate how well each model output matches the reference based on the rubric.
2. Give **specific feedback** for each output using the rubric criteria.
3. Assign a score between 1 and 5 to each output.
4. Finally, say which output is better, or if it's a tie.

---

### Question / Instruction:
{question}

### Ground Truth (Reference):
{ground_truth}

### Baseline Output (Without RAG):
{answer_without_rag}

### RAG Output (With RAG):
{answer_with_rag}

---

### Evaluation Rubric:
Evaluate each output according to:
1. **Dialectal Accuracy**: Does it match the dialectal form expected (e.g., Egyptian, Gulf, etc.)?
2. **Content Relevance**: Does it reflect the core idea in the question and align with the reference?
3. **Fluency & Naturalness**: Does the text read like something a native speaker would say?
4. **Completeness**: Is the output sufficient and covers the expected meaning or intent?

**Scoring scale (for each output)**:
- **1**: Poor — fails most criteria
- **2**: Weak — major issues present
- **3**: Fair — partially aligned, but flawed
- **4**: Good — mostly aligns with the reference
- **5**: Excellent — highly similar, natural, and accurate

---

### Output Format:
Please provide your evaluation in the following structured format:

**BASELINE EVALUATION (Without RAG):**
- **Dialectal Accuracy:** [Brief assessment]
- **Content Relevance:** [Brief assessment]
- **Fluency & Naturalness:** [Brief assessment]
- **Completeness:** [Brief assessment]
- **Overall Feedback:** [Detailed feedback]
- **Score:** [1-5]

**RAG EVALUATION (With RAG):**
- **Dialectal Accuracy:** [Brief assessment]
- **Content Relevance:** [Brief assessment]
- **Fluency & Naturalness:** [Brief assessment]
- **Completeness:** [Brief assessment]
- **Overall Feedback:** [Detailed feedback]
- **Score:** [1-5]

**FINAL COMPARISON:**
- **Better Output:** [Baseline/RAG/Tie]
- **Reasoning:** [Explanation for the choice]
- **Score Difference:** [RAG_Score - Baseline_Score]

Begin your evaluation:"""

    try:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": evaluation_prompt}],
            temperature=0.1,
            max_tokens=2000
        )

        response_text = completion.choices[0].message.content.strip()

        # Use the fixed parsing function
        result = parse_evaluation_response_fixed(response_text)

        return result

    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        return {
            'llm_judge_baseline_score': 0,
            'llm_judge_rag_score': 0,
            'llm_judge_better_output': 'Error',
            'llm_judge_score_difference': 0,
            'llm_judge_baseline_feedback': '',
            'llm_judge_rag_feedback': '',
            'llm_judge_comparison_reasoning': '',
            'llm_judge_raw_evaluation': f"Error: {str(e)}",
            'llm_judge_status': 'failed'
        }

🧪 TESTING PARSING WITH YOUR EXAMPLE:
Baseline Score: 5
RAG Score: 5
Better Output: Baseline
Score Difference: 0
Status: success
Baseline Feedback: إجابة ممتازة ومفصلة تتجاوز المرجع بشكل إيجابي، مما يوفر للقارئ خطوات واضحة وعملية لبدء البيزنس الأون...
RAG Feedback: إجابة ممتازة ومفصلة تتجاوز المرجع بشكل إيجابي، مما يوفر للقارئ خطوات واضحة وعملية لبدء البيزنس الأون...
Reasoning: النسخة الأساسية تقدم معلومات أكثر تفصيلاً وشمولاً وتكتب باللهجة المصرية بشكل أكثر طلاقة وطبيعية، مما...


In [21]:
# Script to fix your existing CSV file with proper parsing

import pandas as pd
import re
from typing import Dict

def fix_existing_llm_judge_results(csv_path: str, output_path: str):
    """
    Fix existing LLM judge results by re-parsing the raw evaluations
    """

    print("🔧 Fixing existing LLM judge results...")

    # Load the CSV
    df = pd.read_csv(csv_path)

    # Check if we have the raw evaluation column
    if 'llm_judge_raw_evaluation' not in df.columns:
        print("❌ No 'llm_judge_raw_evaluation' column found")
        return

    print(f"📊 Found {len(df)} rows to fix")

    # Apply fixed parsing to each row
    fixed_results = []

    for idx, row in df.iterrows():
        if pd.notna(row['llm_judge_raw_evaluation']) and row['llm_judge_raw_evaluation'] != '':
            print(f"🔄 Fixing row {idx + 1}/{len(df)}")

            # Use the fixed parsing function
            parsed_result = parse_evaluation_response_fixed(row['llm_judge_raw_evaluation'])

            # Update the row with fixed values
            fixed_row = row.copy()
            fixed_row['llm_judge_baseline_score'] = parsed_result['llm_judge_baseline_score']
            fixed_row['llm_judge_rag_score'] = parsed_result['llm_judge_rag_score']
            fixed_row['llm_judge_better_output'] = parsed_result['llm_judge_better_output']
            fixed_row['llm_judge_score_difference'] = parsed_result['llm_judge_score_difference']
            fixed_row['llm_judge_baseline_feedback'] = parsed_result['llm_judge_baseline_feedback']
            fixed_row['llm_judge_rag_feedback'] = parsed_result['llm_judge_rag_feedback']
            fixed_row['llm_judge_comparison_reasoning'] = parsed_result['llm_judge_comparison_reasoning']
            fixed_row['llm_judge_status'] = parsed_result['llm_judge_status']

            fixed_results.append(fixed_row)

            print(f"✅ Fixed: Baseline={parsed_result['llm_judge_baseline_score']}, "
                  f"RAG={parsed_result['llm_judge_rag_score']}, "
                  f"Better={parsed_result['llm_judge_better_output']}")
        else:
            # Keep original row if no raw evaluation
            fixed_results.append(row)
            print(f"⏭️ Skipping row {idx + 1} (no raw evaluation)")

    # Create fixed DataFrame
    fixed_df = pd.DataFrame(fixed_results)

    # Save fixed results
    fixed_df.to_csv(output_path, index=False, encoding='utf-8')

    # Generate summary of fixes
    successful_fixes = fixed_df[fixed_df['llm_judge_status'] == 'success']

    print(f"\n📋 FIXING SUMMARY:")
    print(f"Total rows processed: {len(fixed_df)}")
    print(f"Successfully parsed: {len(successful_fixes)}")
    print(f"Parsing failed: {len(fixed_df) - len(successful_fixes)}")

    if len(successful_fixes) > 0:
        baseline_avg = successful_fixes['llm_judge_baseline_score'].mean()
        rag_avg = successful_fixes['llm_judge_rag_score'].mean()
        improvement = rag_avg - baseline_avg

        print(f"\nCORRECTED SCORES:")
        print(f"Average Baseline Score: {baseline_avg:.2f}/5")
        print(f"Average RAG Score: {rag_avg:.2f}/5")
        print(f"Average Improvement: {improvement:+.2f}")

        # Show outcome distribution
        outcomes = successful_fixes['llm_judge_better_output'].value_counts()
        print(f"\nOutcome Distribution:")
        for outcome, count in outcomes.items():
            percentage = count / len(successful_fixes) * 100
            print(f"  {outcome}: {count} ({percentage:.1f}%)")

    print(f"✅ Fixed results saved to: {output_path}")
    return fixed_df

# Simple function to run the fix
def fix_my_csv():
    """Simple function to fix your CSV - just update the paths"""

    # UPDATE THESE PATHS TO MATCH YOUR FILES
    input_csv = "/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/llm_judge_enhanced_results.csv"  # Your current file
    output_csv = "/content/drive/MyDrive/Facebook Crowdtangle Data/All Zied Download Folder/Renamed by Rafi/AraDiaRAG/Datasets/llm_judge_enhanced_results_FIXED.csv"  # Fixed output

    try:
        fixed_df = fix_existing_llm_judge_results(input_csv, output_csv)

        # Show a few examples of the fixed data
        if len(fixed_df) > 0:
            print(f"\n🔍 SAMPLE OF FIXED DATA:")
            sample = fixed_df[fixed_df['llm_judge_status'] == 'success'].head(3)
            for idx, row in sample.iterrows():
                print(f"\nSample {idx + 1}:")
                print(f"  Question: {row['question'][:60]}...")
                print(f"  Baseline Score: {row['llm_judge_baseline_score']}")
                print(f"  RAG Score: {row['llm_judge_rag_score']}")
                print(f"  Better Output: {row['llm_judge_better_output']}")
                print(f"  Score Difference: {row['llm_judge_score_difference']}")

        return fixed_df

    except Exception as e:
        print(f"❌ Error fixing CSV: {e}")
        return None

# If you want to run it directly:
if __name__ == "__main__":
    # Test the parsing function first
    print("🧪 Testing parsing function...")
    test_parsing_with_your_example()

    print("\n" + "="*50)

    # Then fix your actual CSV
    print("🔧 Fixing your CSV file...")
    fixed_data = fix_my_csv()

🧪 Testing parsing function...
🧪 TESTING PARSING WITH YOUR EXAMPLE:
Baseline Score: 5
RAG Score: 5
Better Output: Baseline
Score Difference: 0
Status: success
Baseline Feedback: إجابة ممتازة ومفصلة تتجاوز المرجع بشكل إيجابي، مما يوفر للقارئ خطوات واضحة وعملية لبدء البيزنس الأون...
RAG Feedback: إجابة ممتازة ومفصلة تتجاوز المرجع بشكل إيجابي، مما يوفر للقارئ خطوات واضحة وعملية لبدء البيزنس الأون...
Reasoning: النسخة الأساسية تقدم معلومات أكثر تفصيلاً وشمولاً وتكتب باللهجة المصرية بشكل أكثر طلاقة وطبيعية، مما...

🔧 Fixing your CSV file...
🔧 Fixing existing LLM judge results...
📊 Found 100 rows to fix
🔄 Fixing row 1/100
✅ Fixed: Baseline=2, RAG=2, Better=Tie
🔄 Fixing row 2/100
✅ Fixed: Baseline=1, RAG=1, Better=Tie
🔄 Fixing row 3/100
✅ Fixed: Baseline=1, RAG=1, Better=RAG
🔄 Fixing row 4/100
✅ Fixed: Baseline=4, RAG=4, Better=RAG
🔄 Fixing row 5/100
✅ Fixed: Baseline=2, RAG=2, Better=تعادل
🔄 Fixing row 6/100
✅ Fixed: Baseline=2, RAG=2, Better=RAG
🔄 Fixing row 7/100
✅ Fixed: Baseline=4, RAG=4,